In [1]:
import pandas as pd
import random
import re
from random import randint
from googletrans import Translator
import openai

In [ ]:
# lendo conjunto original para aumentar suas amostras

df = pd.read_csv('Caminho')
df

In [ ]:
# Algoritmo para parafrasear o conjunto de dados com o modelo pré-treinado gpt-3.
# Aceita somente o inglês, utilizado a base de spam.

openai.api_key='Chave'

new = []
sentence = "Rewrite a phrase preserving their original meaning: " 

for i in range (0,232): # Percorre o conjunto de dados
    var = df[df.index == i]
    var = var['Mensagem'].to_string() # Mudar atributo de acordo com o conjunto
    var = re.sub("^\d+\s", "",var).strip()
    
    response = openai.Completion.create(
        model="text-davinci-003",
        prompt=sentence+var,
        temperature=0,
        max_tokens=500,
        frequency_penalty=0.0,
        presence_penalty=0.0
    )
    
    aux = response.choices[0].text[2:]
    new.append(aux)

In [ ]:
# Algoritmo Random Deletion para o conjunto Livraria

new = []

for i in range(0,1163): # Percorre o conjunto de dados
    var = df[df.index == i]
    var = var['Titulo'].to_string() 
    var = re.sub("^\d+\s", "",var).strip()
    spl = var.split()
    if len(spl) > 1:
        if len(spl) <= 5:
            rand = randint(0,len(spl)-1)
            spl.pop(rand)
            aux = ' '.join(spl)
            new.append(aux)
        else:
            for n in range(0,4):
                rand = randint(0,len(spl)-1)
                spl.pop(rand)
            aux = ' '.join(spl)
            new.append(aux)
    else:
        new.append(var)

In [ ]:
# Algoritmo Random Deletion para o conjunto de spam

new = []

for i in range(0,232): # Percorre o conjunto de dados
    var = df[df.index == i]
    var = var['Mensagem'].to_string() 
    var = re.sub("^\d+\s", "",var).strip()
    spl = var.split()
    if len(spl) > 1:
        if len(spl) <= 5:
            rand = randint(0,len(spl)-1)
            spl.pop(rand)
            aux = ' '.join(spl)
            new.append(aux)
        elif len(spl) <= 35:
            for n in range(0,4):
                rand = randint(0,len(spl)-1)
                spl.pop(rand)
            aux = ' '.join(spl)
            new.append(aux)
        else:
            for n in range(0,10):
                rand = randint(0,len(spl)-1)
                spl.pop(rand)
            aux = ' '.join(spl)
            new.append(aux)
    else:
        new.append(var)

In [ ]:
# Algoritmo back-translation

translator = Translator()

new = []

for i in range(0,232): # Percorre o conjunto de dados
    var = df[df.index == i]
    var = var['Titulo'].to_string() # Mudar atributo de acordo com o conjunto
    var = re.sub("^\d+\s", "",var).strip()
    trans1 = translator.translate(var, dest='pt', src='en').text
    trans2 = translator.translate(trans1, dest='en', src='pt').text
    new.append(trans2)

In [ ]:
# Algoritmo Random Swap

new = []

for i in range(0,1163): # tamanho do conjunto
    var = df[df.index == i]
    var = var['Titulo'].to_string() # Mudar atributo de acordo com o conjunto
    var = re.sub("^\d+\s", "",var).strip()
    spl = var.split()
    if len(spl) > 1:
        for n in range(0,2):
            rand1 = randint(0,len(spl)-1)
            rand2 = randint(0,len(spl)-1)
            aux1 = spl.pop(rand1)
            spl.insert(rand2, aux1)
        aux2 = ' '.join(spl)
        new.append(aux2)
    else:
        new.append(var)

In [ ]:
# Depois de aplicado os algoritmos de data augmentation o mesmo é transformado em um novo dataframe

novo = pd.DataFrame(new)
novo['Genero'] = df['Genero'] # Substituir atributo de acordo com dataset
novo.rename(columns={0: 'Titulo'}, inplace = True) 

fim = df.append(novo, ignore_index=True)

# embaralhar as novas amostras
fim = fim.sample(frac = 1)

fim = fim.reset_index(drop=True)

# Salvar o conjunto com as novas amostras
fim.to_csv('Caminho', index=None)